# Open Access status for WoS DOIs

Notebook used for the study presented at the EAHIL 2022 conference

* Title: Computational assistance in the analysis of cited references in biomedical literature: a case study from two institutions.

* Authors:
 * Teresa Lee, Knowledge Manager, International Agency for Research on Cancer (IARC/WHO) leet@iarc.fr  
 * Pablo Iriarte, IT Coordinator, Library of the University of Geneva Pablo.Iriarte@unige.ch 
 * Floriane Muller, Librarian (Medical Library), Library of the University of Geneva Floriane.Muller@unige.ch  
 * Ramon Cierco Jimenez, Doctoral Student, International Agency for Research on Cancer (IARC/WHO) CiercoR@students.iarc.fr  


### Prerequisites
* DOIs from publications and citations
* unpaywall data downloaded using the notebook "doi_unpaywall.ipynb" or queries made directly on unpaywall website

## unpaywall data extraction

https://unpaywall.org/data-format

Fields to extract:

 - is_oa : Boolean (Is there an OA copy of this resource
 - oa_status : Strin (The OA status, or color, of this resource: gold, hybrid, bronze, green or closed)
 - has_repository_copy : Boolean (Whether there is a copy of this resource in a repository. True if this resource has at least one OA Location with host_type = "repository". False otherwise.)




In [1]:
import re
import os
import pandas as pd
import time
import datetime
import json
import numpy as np

## DOIs


In [2]:
# open DOIs data
dois = pd.read_csv('export_dois_all_dedup.tsv', encoding='utf-8', sep='\t', header=0)
dois

,Accession Number,DOI,ID
0,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,1
1,WOS:000579849900001,10.1111/ger.12502,2
2,WOS:000308547000129,10.1111/j.1741-2358.2011.00586.x,3
3,WOS:000331109200007,10.1111/ger.12083,4
4,WOS:000450090700002,10.1016/j.jmpt.2018.01.008,5
...,...,...,...
743544,WOS:000449710900010,10.1016/S0140-6736(08)61939-X,743545
743545,WOS:000449710900010,10.1017/S0033291706009159,743546
743546,WOS:000449710900010,10.1027/0227-5910/a000005,743547
743547,WOS:000449710900010,10.1016/j.aap.2013.11.015,743548


In [3]:
# del and rename cols
del dois['Accession Number']
dois = dois.rename(columns={'DOI' : 'doi', 'ID': 'id'})
dois

,doi,id
0,10.1111/j.1741-2358.2009.00330.x,1
1,10.1111/ger.12502,2
2,10.1111/j.1741-2358.2011.00586.x,3
3,10.1111/ger.12083,4
4,10.1016/j.jmpt.2018.01.008,5
...,...,...
743544,10.1016/S0140-6736(08)61939-X,743545
743545,10.1017/S0033291706009159,743546
743546,10.1027/0227-5910/a000005,743547
743547,10.1016/j.aap.2013.11.015,743548


In [4]:
# open the second series of DOIs (obtained after fixing the bug on the parsing code)
dois_2 = pd.read_csv('export_dois_without_unpaywall.tsv', encoding='utf-8', sep='\t', header=0)
dois_2

,id,doi
0,800001,10.23804/ejpd.2019.20.01.11
1,800002,10.3290/j.qi.a30771
2,800003,10.3290/j.jad.a27795
3,800004,10.3290/j.qi.a44635
4,800005,10.3290/j.qi.a29751
...,...,...
11121,811122,10.1079/PAVSNNR20094005
11122,811123,"[10.1016/j.atherosclerosis.2019.08.014, 10.109..."
11123,811124,10.1101/326470V6
11124,811125,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-..."


In [5]:
# append
dois = dois.append(dois_2, ignore_index=True)
dois

C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,doi,id
0,10.1111/j.1741-2358.2009.00330.x,1
1,10.1111/ger.12502,2
2,10.1111/j.1741-2358.2011.00586.x,3
3,10.1111/ger.12083,4
4,10.1016/j.jmpt.2018.01.008,5
...,...,...
754670,10.1079/PAVSNNR20094005,811122
754671,"[10.1016/j.atherosclerosis.2019.08.014, 10.109...",811123
754672,10.1101/326470V6,811124
754673,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",811125


In [6]:
# dedup by DOI and keep only the last
dois = dois.drop_duplicates(subset='doi', keep='last')
dois

,doi,id
0,10.1111/j.1741-2358.2009.00330.x,1
1,10.1111/ger.12502,2
2,10.1111/j.1741-2358.2011.00586.x,3
3,10.1111/ger.12083,4
4,10.1016/j.jmpt.2018.01.008,5
...,...,...
754670,10.1079/PAVSNNR20094005,811122
754671,"[10.1016/j.atherosclerosis.2019.08.014, 10.109...",811123
754672,10.1101/326470V6,811124
754673,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",811125


In [7]:
# export to csv and excel
dois.to_csv('export_dois_1_and_2_dedup.tsv', sep='\t', encoding='utf-8', index=False)
dois.to_excel('export_dois_1_and_2_dedup.xlsx', index=False)

## Publications from UNIGE


In [8]:
# open UNIGE data
unige_citations = pd.read_csv('WoS UNIGE results 2001_2020/UNIGE_WoS.csv', encoding='utf-8', header=0, usecols=['ID', 'Accession Number', 'DOI', 'DOI of cited article'])
unige_citations

,ID,Accession Number,DOI,DOI of cited article
0,0,WOS:000307585400002,10.1111/j.1741-2358.2011.00508.x,NaN
1,1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,NaN
2,1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1111/j.1600-0528.1999.tb02031.x
3,1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1177/00220345990780020301
4,1,WOS:000287362000002,10.1111/j.1741-2358.2009.00330.x,10.1111/j.1741-2358.2001.00079.x
...,...,...,...,...
1260976,36372,WOS:000591530900014,10.1093/chemse/bjaa041,NaN
1260977,36372,WOS:000591530900014,10.1093/chemse/bjaa041,10.1002/alr.22577
1260978,36372,WOS:000591530900014,10.1093/chemse/bjaa041,10.1002/alr.22592
1260979,36372,WOS:000591530900014,10.1093/chemse/bjaa041,10.1109/TPAMI.2020.2975798


In [9]:
unige_citations.loc[unige_citations['Accession Number'].isna()]

,ID,Accession Number,DOI,DOI of cited article


In [10]:
unige_citations.loc[unige_citations['Accession Number']=='']

,ID,Accession Number,DOI,DOI of cited article


In [11]:
# unige publications by ID
unige_publications_rows = unige_citations[['ID', 'Accession Number', 'DOI']].drop_duplicates(subset='ID').shape[0]
unige_publications_rows

36373

In [12]:
# unige publications
unige_publications = unige_citations[['Accession Number', 'DOI']].drop_duplicates(subset='Accession Number')
del unige_publications['Accession Number']
unige_publications = unige_publications.rename(columns={'DOI' : 'doi'})
unige_publications_dois = unige_publications.loc[unige_publications['doi'].notna()]
unige_publications_dois

,doi
0,10.1111/j.1741-2358.2011.00508.x
1,10.1111/j.1741-2358.2009.00330.x
48,10.1111/ger.12502
83,10.1111/j.1741-2358.2011.00586.x
119,10.1111/ger.12083
...,...
1260758,10.1111/bdi.12659
1260790,10.21037/atm-20-3754
1260869,10.21037/atm-2020-24
1260870,10.1016/j.jvs.2020.06.112


In [13]:
unige_publications

,doi
0,10.1111/j.1741-2358.2011.00508.x
1,10.1111/j.1741-2358.2009.00330.x
48,10.1111/ger.12502
83,10.1111/j.1741-2358.2011.00586.x
119,10.1111/ger.12083
...,...
1260758,10.1111/bdi.12659
1260790,10.21037/atm-20-3754
1260869,10.21037/atm-2020-24
1260870,10.1016/j.jvs.2020.06.112


In [14]:
# unige citations with DOIs
del unige_citations['Accession Number']
del unige_citations['ID']
del unige_citations['DOI']
unige_citations = unige_citations.rename(columns={'DOI of cited article' : 'doi'})
unige_citations_dois = unige_citations.loc[unige_citations['doi'].notna()]
unige_citations_dois

,doi
2,10.1111/j.1600-0528.1999.tb02031.x
3,10.1177/00220345990780020301
4,10.1111/j.1741-2358.2001.00079.x
5,10.1161/01.STR.27.10.1812
6,10.1161/01.STR.24.10.1478
...,...
1260975,10.1007/s00415-010-5763-5
1260977,10.1002/alr.22577
1260978,10.1002/alr.22592
1260979,10.1109/TPAMI.2020.2975798


In [15]:
# open IARC data
iarc_citations = pd.read_csv('WoS IARC results 2001_2020/IARC_WoS.csv', encoding='utf-8', header=0, usecols=['Accession Number', 'DOI', 'DOI of cited article'])
iarc_citations

,Accession Number,DOI,DOI of cited article
0,WOS:000492162206048,NaN,NaN
1,WOS:000438342200039,10.1093/ije/dyy122,10.1093/ije/dyy063
2,WOS:000443381100007,10.19191/EP18.1.P009.008,10.1093/ije/dyg257
3,WOS:000443381100007,10.19191/EP18.1.P009.008,NaN
4,WOS:000443381100007,10.19191/EP18.1.P009.008,NaN
...,...,...,...
275617,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275618,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275619,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN
275620,WOS:000449710900010,10.1016/S0140-6736(18)32281-5,NaN


In [16]:
# iarc publications with DOIs
iarc_publications = iarc_citations[['Accession Number', 'DOI']].drop_duplicates(subset='Accession Number')
del iarc_publications['Accession Number']
iarc_publications = iarc_publications.rename(columns={'DOI' : 'doi'})
iarc_publications_dois = iarc_publications.loc[iarc_publications['doi'].notna()]
iarc_publications_dois

,doi
1,10.1093/ije/dyy122
2,10.19191/EP18.1.P009.008
5,10.1016/S1470-2045(17)30832-X
20,10.1007/s40944-017-0100-x
48,10.1016/j.toxlet.2016.07.082
...,...
275178,10.1016/S0140-6736(15)00128-2
275264,10.1016/S0140-6736(17)32129-3
275318,10.1038/s41586-019-1171-x
275376,10.1016/S0140-6736(16)31012-1


In [17]:
# iarc citations with DOIs
del iarc_citations['Accession Number']
del iarc_citations['DOI']
iarc_citations = iarc_citations.rename(columns={'DOI of cited article' : 'doi'})
iarc_citations_dois = iarc_citations.loc[iarc_citations['doi'].notna()]
iarc_citations_dois

,doi
1,10.1093/ije/dyy063
2,10.1093/ije/dyg257
5,10.1093/carcin/bgq074
7,10.1006/faat.1993.1061
8,10.1093/aje/kwv111
...,...
275600,10.1371/journal.pmed.1001242]
275601,10.1080/03630242.2011.637611
275610,"[10.1016/S0140-6736(16)31012-1, 10.1016/S0140-..."
275616,10.1046/j.1360-0443.2000.95225112.x


In [18]:
# add columns
unige_publications_dois['doi'] = unige_publications_dois['doi'].str.strip()
unige_citations_dois['doi'] = unige_citations_dois['doi'].str.strip()
iarc_publications_dois['doi'] = iarc_publications_dois['doi'].str.strip()
iarc_citations_dois['doi'] = iarc_citations_dois['doi'].str.strip()
unige_publications_dois['publication_unige'] = 1
unige_citations_dois['citation_unige'] = 1
iarc_publications_dois['publication_iarc'] = 1
iarc_citations_dois['citation_iarc'] = 1

C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a c

In [19]:
# merge the publication and citation information
dois = pd.merge(dois, unige_publications_dois, on='doi', how='left')
dois = pd.merge(dois, unige_citations_dois, on='doi', how='left')
dois = pd.merge(dois, iarc_publications_dois, on='doi', how='left')
dois = pd.merge(dois, iarc_citations_dois, on='doi', how='left')
dois

,doi,id,publication_unige,citation_unige,publication_iarc,citation_iarc
0,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN
1,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN
2,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN
3,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN
4,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...
1326233,10.1079/PAVSNNR20094005,811122,NaN,NaN,NaN,1.0
1326234,"[10.1016/j.atherosclerosis.2019.08.014, 10.109...",811123,NaN,NaN,NaN,1.0
1326235,10.1101/326470V6,811124,NaN,NaN,NaN,1.0
1326236,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",811125,NaN,NaN,NaN,1.0


In [20]:
# drop duplicates
dois = dois.drop_duplicates(subset='id')
dois

,doi,id,publication_unige,citation_unige,publication_iarc,citation_iarc
0,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN
7,10.1111/ger.12502,2,1.0,NaN,NaN,NaN
8,10.1111/j.1741-2358.2011.00586.x,3,1.0,1.0,NaN,NaN
16,10.1111/ger.12083,4,1.0,1.0,NaN,NaN
22,10.1016/j.jmpt.2018.01.008,5,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...
1326233,10.1079/PAVSNNR20094005,811122,NaN,NaN,NaN,1.0
1326234,"[10.1016/j.atherosclerosis.2019.08.014, 10.109...",811123,NaN,NaN,NaN,1.0
1326235,10.1101/326470V6,811124,NaN,NaN,NaN,1.0
1326236,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",811125,NaN,NaN,NaN,1.0


In [21]:
# reset index
dois.reset_index(drop=True, inplace=True)
dois

,doi,id,publication_unige,citation_unige,publication_iarc,citation_iarc
0,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN
1,10.1111/ger.12502,2,1.0,NaN,NaN,NaN
2,10.1111/j.1741-2358.2011.00586.x,3,1.0,1.0,NaN,NaN
3,10.1111/ger.12083,4,1.0,1.0,NaN,NaN
4,10.1016/j.jmpt.2018.01.008,5,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...
744564,10.1079/PAVSNNR20094005,811122,NaN,NaN,NaN,1.0
744565,"[10.1016/j.atherosclerosis.2019.08.014, 10.109...",811123,NaN,NaN,NaN,1.0
744566,10.1101/326470V6,811124,NaN,NaN,NaN,1.0
744567,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",811125,NaN,NaN,NaN,1.0


In [22]:
# test empty rows
dois.loc[dois['publication_unige'].isna() & dois['citation_unige'].isna() & dois['publication_iarc'].isna() & dois['citation_iarc'].isna()]

,doi,id,publication_unige,citation_unige,publication_iarc,citation_iarc


In [23]:
# export
dois.to_csv('export_dois_all_dedup_publications_citations.tsv', sep='\t', index=False)

## unpaywall OA status

In [22]:
# extract informtation on downloaded data from unpaywall
for index, row in dois.iterrows():
    mydoi = row['doi']
    myid = str(row['id']).zfill(10)
    myfolder = str(int(row['id']/100000)+1)
    # print(myid)
    if (((index/1000) - int(index/1000)) == 0) :
        print(index)
    # open the json file
    if os.path.exists('E:/data_sources/unpaywall/eahil_2022/eahil_2022_' + myfolder + '/' + myid + '.json'):
        # initialising variables
        doi_unpaywall = ''
        oa_status = ''
        has_repository_copy = ''
        with open('E:/data_sources/unpaywall/eahil_2022/eahil_2022_' + myfolder + '/' + myid + '.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
            if ('doi' in data):
                doi_unpaywall = data['doi']
            if ('oa_status' in data):
                oa_status = data['oa_status']
            if ('has_repository_copy' in data):
                has_repository_copy = data['has_repository_copy']

        dois.at[index,'doi_unpaywall'] = doi_unpaywall
        dois.at[index,'oa_status'] = oa_status
        dois.at[index,'has_repository_copy'] = has_repository_copy
    else :
        # print(str(row['id']) + ' - not found')
        with open('unpaywall_extract_errors.txt', 'a', encoding='utf-8') as g:
            g.write(str(row['id']) + '	file not found\n')

0


C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [23]:
dois

,doi,id,publication_unige,citation_unige,publication_iarc,citation_iarc,doi_unpaywall,oa_status,has_repository_copy
0,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2009.00330.x,closed,False
1,10.1111/ger.12502,2,1.0,NaN,NaN,NaN,10.1111/ger.12502,closed,False
2,10.1111/j.1741-2358.2011.00586.x,3,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2011.00586.x,bronze,True
3,10.1111/ger.12083,4,1.0,1.0,NaN,NaN,10.1111/ger.12083,closed,False
4,10.1016/j.jmpt.2018.01.008,5,1.0,NaN,NaN,NaN,10.1016/j.jmpt.2018.01.008,closed,False
...,...,...,...,...,...,...,...,...,...
751910,10.1079/PAVSNNR20094005,818500,NaN,NaN,NaN,1.0,NaN,NaN,NaN
751911,"[10.1016/j.atherosclerosis.2019.08.014, 10.109...",818501,NaN,NaN,NaN,1.0,NaN,NaN,NaN
751912,10.1101/326470V6,818502,NaN,NaN,NaN,1.0,NaN,NaN,NaN
751913,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",818503,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [24]:
dois['oa_status'].value_counts()

closed    406553
bronze    169678
green     100580
hybrid     37266
gold       26712
Name: oa_status, dtype: int64

In [25]:
dois['has_repository_copy'].value_counts()

False    518033
True     222756
Name: has_repository_copy, dtype: int64

In [26]:
# dois found
dois.loc[dois['doi_unpaywall'].notna()].shape[0]

740789

In [27]:
# dois found
dois.loc[dois['doi_unpaywall'].notna()].shape[0] / dois.shape[0]

0.9852031147137642

In [28]:
# dois not found
dois.loc[dois['doi_unpaywall'].isna()].shape[0]

11126

In [29]:
# dois not found
dois.loc[dois['doi_unpaywall'].isna()].shape[0] / dois.shape[0]

0.014796885286235812

In [30]:
# dois not equal
dois.loc[(dois['doi'] != dois['doi_unpaywall']) & dois['doi_unpaywall'].notna()]

,doi,id,publication_unige,citation_unige,publication_iarc,citation_iarc,doi_unpaywall,oa_status,has_repository_copy
29,10.1017/S0266462318000260,30,1.0,1.0,NaN,NaN,10.1017/s0266462318000260,closed,False
44,10.1016/S0020-1383(15)70003-3,45,1.0,NaN,NaN,NaN,10.1016/s0020-1383(15)70003-3,closed,False
50,10.1017/S0266462314000233,51,1.0,NaN,NaN,NaN,10.1017/s0266462314000233,hybrid,True
59,10.1097/FPC.0b013e3280925716,61,1.0,1.0,NaN,NaN,10.1097/fpc.0b013e3280925716,closed,False
60,10.1097/FPC.0000000000000313,62,1.0,NaN,NaN,NaN,10.1097/fpc.0000000000000313,green,True
...,...,...,...,...,...,...,...,...,...
749945,"[10.1016/j.patol.2017.12.002, 10.1016/J.PATOL....",816535,NaN,NaN,NaN,1.0,10.1016/j.patol.2017.12.002,closed,False
749946,"[10.12809/hkmj164885, 10.12809/hkmj176808]",816536,NaN,NaN,NaN,1.0,10.12809/hkmj164885,gold,True
749947,"[10.1097/qad.0000000000001765, 10.1097/QAD.000...",816537,NaN,NaN,NaN,1.0,10.1097/qad.0000000000001765,green,True
749949,"[10.1016/j.vaccine.2013.07.026, 10.1016/j.vacc...",816539,NaN,NaN,NaN,1.0,10.1016/j.vaccine.2013.07.026,green,True


In [31]:
# normalize DOIs
dois.loc[dois['doi'].notna(), 'doi_normalized'] = dois['doi'].str.upper()
dois.loc[dois['doi_unpaywall'].notna(), 'doi_unpaywall_normalized'] = dois['doi_unpaywall'].str.upper()
dois

,doi,id,publication_unige,citation_unige,publication_iarc,citation_iarc,doi_unpaywall,oa_status,has_repository_copy,doi_normalized,doi_unpaywall_normalized
0,10.1111/j.1741-2358.2009.00330.x,1,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2009.00330.x,closed,False,10.1111/J.1741-2358.2009.00330.X,10.1111/J.1741-2358.2009.00330.X
1,10.1111/ger.12502,2,1.0,NaN,NaN,NaN,10.1111/ger.12502,closed,False,10.1111/GER.12502,10.1111/GER.12502
2,10.1111/j.1741-2358.2011.00586.x,3,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2011.00586.x,bronze,True,10.1111/J.1741-2358.2011.00586.X,10.1111/J.1741-2358.2011.00586.X
3,10.1111/ger.12083,4,1.0,1.0,NaN,NaN,10.1111/ger.12083,closed,False,10.1111/GER.12083,10.1111/GER.12083
4,10.1016/j.jmpt.2018.01.008,5,1.0,NaN,NaN,NaN,10.1016/j.jmpt.2018.01.008,closed,False,10.1016/J.JMPT.2018.01.008,10.1016/J.JMPT.2018.01.008
...,...,...,...,...,...,...,...,...,...,...,...
751910,10.1079/PAVSNNR20094005,818500,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.1079/PAVSNNR20094005,NaN
751911,"[10.1016/j.atherosclerosis.2019.08.014, 10.109...",818501,NaN,NaN,NaN,1.0,NaN,NaN,NaN,"[10.1016/J.ATHEROSCLEROSIS.2019.08.014, 10.109...",NaN
751912,10.1101/326470V6,818502,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.1101/326470V6,NaN
751913,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",818503,NaN,NaN,NaN,1.0,NaN,NaN,NaN,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",NaN


In [32]:
# dois not equal
dois.loc[(dois['doi_normalized'] != dois['doi_unpaywall_normalized']) & dois['doi_unpaywall_normalized'].notna()]

,doi,id,publication_unige,citation_unige,publication_iarc,citation_iarc,doi_unpaywall,oa_status,has_repository_copy,doi_normalized,doi_unpaywall_normalized
85157,10.1126/science. 1068094,85952,NaN,1.0,NaN,NaN,10.1126/science.,closed,False,10.1126/SCIENCE. 1068094,10.1126/SCIENCE.
122587,10.2307/1293082.,123860,NaN,1.0,NaN,NaN,10.2307/1293082,closed,False,10.2307/1293082.,10.2307/1293082
122719,10.1101/gr.206602. Article published online be...,123993,NaN,1.0,NaN,NaN,10.1101/gr.206602.,hybrid,True,10.1101/GR.206602. ARTICLE PUBLISHED ONLINE BE...,10.1101/GR.206602.
146668,10.1126/science. 1140516,148235,NaN,1.0,NaN,NaN,10.1126/science.,closed,False,10.1126/SCIENCE. 1140516,10.1126/SCIENCE.
188852,10.1038/S41467-019-11513-1),191048,NaN,1.0,NaN,NaN,10.1038/s41467-019-11513-1,gold,True,10.1038/S41467-019-11513-1),10.1038/S41467-019-11513-1
...,...,...,...,...,...,...,...,...,...,...,...
749945,"[10.1016/j.patol.2017.12.002, 10.1016/J.PATOL....",816535,NaN,NaN,NaN,1.0,10.1016/j.patol.2017.12.002,closed,False,"[10.1016/J.PATOL.2017.12.002, 10.1016/J.PATOL....",10.1016/J.PATOL.2017.12.002
749946,"[10.12809/hkmj164885, 10.12809/hkmj176808]",816536,NaN,NaN,NaN,1.0,10.12809/hkmj164885,gold,True,"[10.12809/HKMJ164885, 10.12809/HKMJ176808]",10.12809/HKMJ164885
749947,"[10.1097/qad.0000000000001765, 10.1097/QAD.000...",816537,NaN,NaN,NaN,1.0,10.1097/qad.0000000000001765,green,True,"[10.1097/QAD.0000000000001765, 10.1097/QAD.000...",10.1097/QAD.0000000000001765
749949,"[10.1016/j.vaccine.2013.07.026, 10.1016/j.vacc...",816539,NaN,NaN,NaN,1.0,10.1016/j.vaccine.2013.07.026,green,True,"[10.1016/J.VACCINE.2013.07.026, 10.1016/J.VACC...",10.1016/J.VACCINE.2013.07.026


In [33]:
# dois not equal
dois.loc[(dois['doi_normalized'] != dois['doi_unpaywall_normalized']) & dois['doi_unpaywall_normalized'].notna()].shape[0]

3706

In [34]:
# export dois not equal
dois.loc[(dois['doi_normalized'] != dois['doi_unpaywall_normalized']) & dois['doi_unpaywall_normalized'].notna()].to_excel('export_dois_wos_unpaywall_not_equal.xlsx', index=False)

In [35]:
# export to csv and excel
dois.to_csv('export_dois_oa_status.tsv', sep='\t', encoding='utf-8', index=False)
dois.to_excel('export_dois_oa_status.xlsx', index=False)

# Add OA status to publications and citations

In [36]:
# open UNIGE data
unige_all = pd.read_csv('WoS UNIGE results 2001_2020/UNIGE_WoS.csv', encoding='utf-8', header=0, usecols=['Accession Number', 'Document Type', 'Year Published', '29 Character Source Abbreviation', 'Open Access Indicator', 'PubMed ID', 'DOI', 'Year of cited article', 'Journal of cited article', 'DOI of cited article'])
unige_all

C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Accession Number,Year Published,Document Type,29 Character Source Abbreviation,DOI,PubMed ID,Open Access Indicator,Year of cited article,Journal of cited article,DOI of cited article
0,WOS:000307585400002,2012,Article,GERODONTOLOGY,10.1111/j.1741-2358.2011.00508.x,22591169,NaN,NaN,NaN,NaN
1,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2002.0,INT J PROSTHODONT,NaN
2,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,COMMUNITY DENT ORAL,10.1111/j.1600-0528.1999.tb02031.x
3,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,J DENT RES,10.1177/00220345990780020301
4,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2001.0,Gerodontology,10.1111/j.1741-2358.2001.00079.x
...,...,...,...,...,...,...,...,...,...,...
1260976,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,Q A COR COVID 19,NaN
1260977,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22577
1260978,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22592
1260979,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2021.0,IEEE T PATTERN ANAL,10.1109/TPAMI.2020.2975798


In [37]:
# open IARC data
iarc_all = pd.read_csv('WoS IARC results 2001_2020/IARC_WoS.csv', encoding='utf-8', header=0, usecols=['Accession Number', 'Document Type', 'Year Published', '29 Character Source Abbreviation', 'Open Access Indicator', 'PubMed ID', 'DOI', 'Year of cited article', 'Journal of cited article', 'DOI of cited article'])
iarc_all

,Accession Number,Year Published,Document Type,29 Character Source Abbreviation,DOI,PubMed ID,Open Access Indicator,Year of cited article,Journal of cited article,DOI of cited article
0,WOS:000492162206048,2019,Meeting Abstract,J THORAC ONCOL,NaN,NaN,NaN,NaN,NaN,NaN
1,WOS:000438342200039,2018,Letter,INT J EPIDEMIOL,10.1093/ije/dyy122,NaN,Bronze,2018.0,INT J EPIDEMIOL,10.1093/ije/dyy063
2,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2003.0,INT J EPIDEMIOL,10.1093/ije/dyg257
3,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2017.0,R P,NaN
4,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2016.0,LISTEN LIBERAL WHAT,NaN
...,...,...,...,...,...,...,...,...,...,...
275617,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,ATL SUST DEV GOALS 2,NaN
275618,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2011.0,BURD BEST BUYS RED E,NaN
275619,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2006.0,WORK TOG HLTH WORLD,NaN
275620,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,WORLD HLTH STAT 2018,NaN


In [38]:
# rename columns
unige_all = unige_all.rename(columns={'Accession Number' : 'publication_wos_id',
                                      'Year Published' : 'publication_year',
                                     'Document Type' : 'publication_type',
                                     '29 Character Source Abbreviation' : 'publication_journal',
                                     'DOI' : 'publication_doi',
                                     'PubMed ID' : 'publication_pmid',
                                     'Open Access Indicator' : 'publication_wos_oa',
                                     'Year of cited article' : 'citation_year',
                                     'Journal of cited article' : 'citation_journal',
                                     'DOI of cited article' : 'citation_doi'})
unige_all

,publication_wos_id,publication_year,publication_type,publication_journal,publication_doi,publication_pmid,publication_wos_oa,citation_year,citation_journal,citation_doi
0,WOS:000307585400002,2012,Article,GERODONTOLOGY,10.1111/j.1741-2358.2011.00508.x,22591169,NaN,NaN,NaN,NaN
1,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2002.0,INT J PROSTHODONT,NaN
2,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,COMMUNITY DENT ORAL,10.1111/j.1600-0528.1999.tb02031.x
3,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,J DENT RES,10.1177/00220345990780020301
4,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2001.0,Gerodontology,10.1111/j.1741-2358.2001.00079.x
...,...,...,...,...,...,...,...,...,...,...
1260976,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,Q A COR COVID 19,NaN
1260977,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22577
1260978,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22592
1260979,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2021.0,IEEE T PATTERN ANAL,10.1109/TPAMI.2020.2975798


In [39]:
# rename columns
iarc_all = iarc_all.rename(columns={'Accession Number' : 'publication_wos_id',
                                      'Year Published' : 'publication_year',
                                     'Document Type' : 'publication_type',
                                     '29 Character Source Abbreviation' : 'publication_journal',
                                     'DOI' : 'publication_doi',
                                     'PubMed ID' : 'publication_pmid',
                                     'Open Access Indicator' : 'publication_wos_oa',
                                     'Year of cited article' : 'citation_year',
                                     'Journal of cited article' : 'citation_journal',
                                     'DOI of cited article' : 'citation_doi'})
iarc_all

,publication_wos_id,publication_year,publication_type,publication_journal,publication_doi,publication_pmid,publication_wos_oa,citation_year,citation_journal,citation_doi
0,WOS:000492162206048,2019,Meeting Abstract,J THORAC ONCOL,NaN,NaN,NaN,NaN,NaN,NaN
1,WOS:000438342200039,2018,Letter,INT J EPIDEMIOL,10.1093/ije/dyy122,NaN,Bronze,2018.0,INT J EPIDEMIOL,10.1093/ije/dyy063
2,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2003.0,INT J EPIDEMIOL,10.1093/ije/dyg257
3,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2017.0,R P,NaN
4,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2016.0,LISTEN LIBERAL WHAT,NaN
...,...,...,...,...,...,...,...,...,...,...
275617,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,ATL SUST DEV GOALS 2,NaN
275618,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2011.0,BURD BEST BUYS RED E,NaN
275619,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2006.0,WORK TOG HLTH WORLD,NaN
275620,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,WORLD HLTH STAT 2018,NaN


In [40]:
# strip DOIs
unige_all['publication_doi'] = unige_all['publication_doi'].str.strip()
unige_all['citation_doi'] = unige_all['citation_doi'].str.strip()
iarc_all['publication_doi'] = iarc_all['publication_doi'].str.strip()
iarc_all['citation_doi'] = iarc_all['citation_doi'].str.strip()

In [41]:
# merge in UNIGE file for publications
unige_all = pd.merge(unige_all, dois, left_on='publication_doi', right_on='doi', how='left')
unige_all

,publication_wos_id,publication_year,publication_type,publication_journal,publication_doi,publication_pmid,publication_wos_oa,citation_year,citation_journal,citation_doi,...,id,publication_unige,citation_unige,publication_iarc,citation_iarc,doi_unpaywall,oa_status,has_repository_copy,doi_normalized,doi_unpaywall_normalized
0,WOS:000307585400002,2012,Article,GERODONTOLOGY,10.1111/j.1741-2358.2011.00508.x,22591169,NaN,NaN,NaN,NaN,...,58351.0,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2011.00508.x,closed,False,10.1111/J.1741-2358.2011.00508.X,10.1111/J.1741-2358.2011.00508.X
1,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2002.0,INT J PROSTHODONT,NaN,...,1.0,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2009.00330.x,closed,False,10.1111/J.1741-2358.2009.00330.X,10.1111/J.1741-2358.2009.00330.X
2,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,COMMUNITY DENT ORAL,10.1111/j.1600-0528.1999.tb02031.x,...,1.0,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2009.00330.x,closed,False,10.1111/J.1741-2358.2009.00330.X,10.1111/J.1741-2358.2009.00330.X
3,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,J DENT RES,10.1177/00220345990780020301,...,1.0,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2009.00330.x,closed,False,10.1111/J.1741-2358.2009.00330.X,10.1111/J.1741-2358.2009.00330.X
4,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2001.0,Gerodontology,10.1111/j.1741-2358.2001.00079.x,...,1.0,1.0,1.0,NaN,NaN,10.1111/j.1741-2358.2009.00330.x,closed,False,10.1111/J.1741-2358.2009.00330.X,10.1111/J.1741-2358.2009.00330.X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260976,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,Q A COR COVID 19,NaN,...,27766.0,1.0,1.0,NaN,NaN,10.1093/chemse/bjaa041,bronze,True,10.1093/CHEMSE/BJAA041,10.1093/CHEMSE/BJAA041
1260977,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22577,...,27766.0,1.0,1.0,NaN,NaN,10.1093/chemse/bjaa041,bronze,True,10.1093/CHEMSE/BJAA041,10.1093/CHEMSE/BJAA041
1260978,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22592,...,27766.0,1.0,1.0,NaN,NaN,10.1093/chemse/bjaa041,bronze,True,10.1093/CHEMSE/BJAA041,10.1093/CHEMSE/BJAA041
1260979,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2021.0,IEEE T PATTERN ANAL,10.1109/TPAMI.2020.2975798,...,27766.0,1.0,1.0,NaN,NaN,10.1093/chemse/bjaa041,bronze,True,10.1093/CHEMSE/BJAA041,10.1093/CHEMSE/BJAA041


In [42]:
# merge in UNIGE file for citations
unige_all = pd.merge(unige_all, dois, left_on='citation_doi', right_on='doi', how='left')
unige_all

,publication_wos_id,publication_year,publication_type,publication_journal,publication_doi,publication_pmid,publication_wos_oa,citation_year,citation_journal,citation_doi,...,id_y,publication_unige_y,citation_unige_y,publication_iarc_y,citation_iarc_y,doi_unpaywall_y,oa_status_y,has_repository_copy_y,doi_normalized_y,doi_unpaywall_normalized_y
0,WOS:000307585400002,2012,Article,GERODONTOLOGY,10.1111/j.1741-2358.2011.00508.x,22591169,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2002.0,INT J PROSTHODONT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,COMMUNITY DENT ORAL,10.1111/j.1600-0528.1999.tb02031.x,...,33855.0,NaN,1.0,NaN,NaN,10.1111/j.1600-0528.1999.tb02031.x,closed,False,10.1111/J.1600-0528.1999.TB02031.X,10.1111/J.1600-0528.1999.TB02031.X
3,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,J DENT RES,10.1177/00220345990780020301,...,33856.0,NaN,1.0,NaN,NaN,10.1177/00220345990780020301,closed,False,10.1177/00220345990780020301,10.1177/00220345990780020301
4,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2001.0,Gerodontology,10.1111/j.1741-2358.2001.00079.x,...,33857.0,NaN,1.0,NaN,NaN,10.1111/j.1741-2358.2001.00079.x,closed,False,10.1111/J.1741-2358.2001.00079.X,10.1111/J.1741-2358.2001.00079.X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260976,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,Q A COR COVID 19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1260977,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22577,...,659661.0,NaN,1.0,NaN,NaN,10.1002/alr.22577,closed,False,10.1002/ALR.22577,10.1002/ALR.22577
1260978,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22592,...,659662.0,NaN,1.0,NaN,NaN,10.1002/alr.22592,bronze,True,10.1002/ALR.22592,10.1002/ALR.22592
1260979,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2021.0,IEEE T PATTERN ANAL,10.1109/TPAMI.2020.2975798,...,659663.0,NaN,1.0,NaN,NaN,10.1109/tpami.2020.2975798,green,True,10.1109/TPAMI.2020.2975798,10.1109/TPAMI.2020.2975798


In [43]:
unige_all.columns

Index(['publication_wos_id', 'publication_year', 'publication_type',
       'publication_journal', 'publication_doi', 'publication_pmid',
       'publication_wos_oa', 'citation_year', 'citation_journal',
       'citation_doi', 'doi_x', 'id_x', 'publication_unige_x',
       'citation_unige_x', 'publication_iarc_x', 'citation_iarc_x',
       'doi_unpaywall_x', 'oa_status_x', 'has_repository_copy_x',
       'doi_normalized_x', 'doi_unpaywall_normalized_x', 'doi_y', 'id_y',
       'publication_unige_y', 'citation_unige_y', 'publication_iarc_y',
       'citation_iarc_y', 'doi_unpaywall_y', 'oa_status_y',
       'has_repository_copy_y', 'doi_normalized_y',
       'doi_unpaywall_normalized_y'],
      dtype='object')

In [44]:
# merge in IARC file
iarc_all = pd.merge(iarc_all, dois, left_on='publication_doi', right_on='doi', how='left')
iarc_all

,publication_wos_id,publication_year,publication_type,publication_journal,publication_doi,publication_pmid,publication_wos_oa,citation_year,citation_journal,citation_doi,...,id,publication_unige,citation_unige,publication_iarc,citation_iarc,doi_unpaywall,oa_status,has_repository_copy,doi_normalized,doi_unpaywall_normalized
0,WOS:000492162206048,2019,Meeting Abstract,J THORAC ONCOL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WOS:000438342200039,2018,Letter,INT J EPIDEMIOL,10.1093/ije/dyy122,NaN,Bronze,2018.0,INT J EPIDEMIOL,10.1093/ije/dyy063,...,27767.0,NaN,NaN,1.0,NaN,10.1093/ije/dyy122,bronze,False,10.1093/IJE/DYY122,10.1093/IJE/DYY122
2,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2003.0,INT J EPIDEMIOL,10.1093/ije/dyg257,...,815391.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,10.19191/EP18.1.P009.008,NaN
3,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2017.0,R P,NaN,...,815391.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,10.19191/EP18.1.P009.008,NaN
4,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2016.0,LISTEN LIBERAL WHAT,NaN,...,815391.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,10.19191/EP18.1.P009.008,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275617,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,ATL SUST DEV GOALS 2,NaN,...,33854.0,NaN,NaN,1.0,NaN,10.1016/s0140-6736(18)32281-5,hybrid,True,10.1016/S0140-6736(18)32281-5,10.1016/S0140-6736(18)32281-5
275618,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2011.0,BURD BEST BUYS RED E,NaN,...,33854.0,NaN,NaN,1.0,NaN,10.1016/s0140-6736(18)32281-5,hybrid,True,10.1016/S0140-6736(18)32281-5,10.1016/S0140-6736(18)32281-5
275619,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2006.0,WORK TOG HLTH WORLD,NaN,...,33854.0,NaN,NaN,1.0,NaN,10.1016/s0140-6736(18)32281-5,hybrid,True,10.1016/S0140-6736(18)32281-5,10.1016/S0140-6736(18)32281-5
275620,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,WORLD HLTH STAT 2018,NaN,...,33854.0,NaN,NaN,1.0,NaN,10.1016/s0140-6736(18)32281-5,hybrid,True,10.1016/S0140-6736(18)32281-5,10.1016/S0140-6736(18)32281-5


In [45]:
# merge in IARC file for citations
iarc_all = pd.merge(iarc_all, dois, left_on='citation_doi', right_on='doi', how='left')
iarc_all

,publication_wos_id,publication_year,publication_type,publication_journal,publication_doi,publication_pmid,publication_wos_oa,citation_year,citation_journal,citation_doi,...,id_y,publication_unige_y,citation_unige_y,publication_iarc_y,citation_iarc_y,doi_unpaywall_y,oa_status_y,has_repository_copy_y,doi_normalized_y,doi_unpaywall_normalized_y
0,WOS:000492162206048,2019,Meeting Abstract,J THORAC ONCOL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WOS:000438342200039,2018,Letter,INT J EPIDEMIOL,10.1093/ije/dyy122,NaN,Bronze,2018.0,INT J EPIDEMIOL,10.1093/ije/dyy063,...,659664.0,NaN,NaN,NaN,1.0,10.1093/ije/dyy063,bronze,False,10.1093/IJE/DYY063,10.1093/IJE/DYY063
2,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2003.0,INT J EPIDEMIOL,10.1093/ije/dyg257,...,10278.0,1.0,1.0,NaN,1.0,10.1093/ije/dyg257,bronze,True,10.1093/IJE/DYG257,10.1093/IJE/DYG257
3,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2017.0,R P,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2016.0,LISTEN LIBERAL WHAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275617,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,ATL SUST DEV GOALS 2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275618,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2011.0,BURD BEST BUYS RED E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275619,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2006.0,WORK TOG HLTH WORLD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275620,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,WORLD HLTH STAT 2018,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
iarc_all.columns

Index(['publication_wos_id', 'publication_year', 'publication_type',
       'publication_journal', 'publication_doi', 'publication_pmid',
       'publication_wos_oa', 'citation_year', 'citation_journal',
       'citation_doi', 'doi_x', 'id_x', 'publication_unige_x',
       'citation_unige_x', 'publication_iarc_x', 'citation_iarc_x',
       'doi_unpaywall_x', 'oa_status_x', 'has_repository_copy_x',
       'doi_normalized_x', 'doi_unpaywall_normalized_x', 'doi_y', 'id_y',
       'publication_unige_y', 'citation_unige_y', 'publication_iarc_y',
       'citation_iarc_y', 'doi_unpaywall_y', 'oa_status_y',
       'has_repository_copy_y', 'doi_normalized_y',
       'doi_unpaywall_normalized_y'],
      dtype='object')

In [47]:
# del columns not used and rename
del unige_all['doi_normalized_x']
del unige_all['doi_unpaywall_normalized_x']
del unige_all['doi_unpaywall_x']
del unige_all['id_x']
del unige_all['doi_x']
del unige_all['doi_normalized_y']
del unige_all['doi_unpaywall_normalized_y']
del unige_all['doi_unpaywall_y']
del unige_all['id_y']
del unige_all['doi_y']
# rename columns
unige_all = unige_all.rename(columns={'publication_unige_x' : 'publication_doi_is_unige_publication',
                                      'publication_unige_y' : 'citation_doi_is_unige_publication',
                                      'citation_unige_x' : 'publication_doi_is_unige_citation',
                                      'citation_unige_y' : 'citation_doi_is_unige_citation',
                                     'publication_iarc_x' : 'publication_doi_is_iarc_publication',
                                      'publication_iarc_y' : 'citation_doi_is_iarc_publication',
                                      'citation_iarc_x' : 'publication_doi_is_iarc_citation',
                                      'citation_iarc_y' : 'citation_doi_is_iarc_citation',
                                     'oa_status_x' : 'publication_doi_oa_status',
                                     'oa_status_y' : 'citation_doi_oa_status',
                                     'has_repository_copy_x' : 'publication_doi_has_repository_copy',
                                     'has_repository_copy_y' : 'citation_doi_has_repository_copy'})
unige_all

,publication_wos_id,publication_year,publication_type,publication_journal,publication_doi,publication_pmid,publication_wos_oa,citation_year,citation_journal,citation_doi,...,publication_doi_is_iarc_publication,publication_doi_is_iarc_citation,publication_doi_oa_status,publication_doi_has_repository_copy,citation_doi_is_unige_publication,citation_doi_is_unige_citation,citation_doi_is_iarc_publication,citation_doi_is_iarc_citation,citation_doi_oa_status,citation_doi_has_repository_copy
0,WOS:000307585400002,2012,Article,GERODONTOLOGY,10.1111/j.1741-2358.2011.00508.x,22591169,NaN,NaN,NaN,NaN,...,NaN,NaN,closed,False,NaN,NaN,NaN,NaN,NaN,NaN
1,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2002.0,INT J PROSTHODONT,NaN,...,NaN,NaN,closed,False,NaN,NaN,NaN,NaN,NaN,NaN
2,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,COMMUNITY DENT ORAL,10.1111/j.1600-0528.1999.tb02031.x,...,NaN,NaN,closed,False,NaN,1.0,NaN,NaN,closed,False
3,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,1999.0,J DENT RES,10.1177/00220345990780020301,...,NaN,NaN,closed,False,NaN,1.0,NaN,NaN,closed,False
4,WOS:000287362000002,2011,Article,GERODONTOLOGY,10.1111/j.1741-2358.2009.00330.x,20337725,NaN,2001.0,Gerodontology,10.1111/j.1741-2358.2001.00079.x,...,NaN,NaN,closed,False,NaN,1.0,NaN,NaN,closed,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260976,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,Q A COR COVID 19,NaN,...,NaN,NaN,bronze,True,NaN,NaN,NaN,NaN,NaN,NaN
1260977,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22577,...,NaN,NaN,bronze,True,NaN,1.0,NaN,NaN,closed,False
1260978,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2020.0,INT FORUM ALLERGY RH,10.1002/alr.22592,...,NaN,NaN,bronze,True,NaN,1.0,NaN,NaN,bronze,True
1260979,WOS:000591530900014,2020,Article,CHEM SENSES,10.1093/chemse/bjaa041,3.25641e+07,"Green Submitted, Green Published, Green Accept...",2021.0,IEEE T PATTERN ANAL,10.1109/TPAMI.2020.2975798,...,NaN,NaN,bronze,True,NaN,1.0,NaN,NaN,green,True


In [48]:
unige_all.columns

Index(['publication_wos_id', 'publication_year', 'publication_type',
       'publication_journal', 'publication_doi', 'publication_pmid',
       'publication_wos_oa', 'citation_year', 'citation_journal',
       'citation_doi', 'publication_doi_is_unige_publication',
       'publication_doi_is_unige_citation',
       'publication_doi_is_iarc_publication',
       'publication_doi_is_iarc_citation', 'publication_doi_oa_status',
       'publication_doi_has_repository_copy',
       'citation_doi_is_unige_publication', 'citation_doi_is_unige_citation',
       'citation_doi_is_iarc_publication', 'citation_doi_is_iarc_citation',
       'citation_doi_oa_status', 'citation_doi_has_repository_copy'],
      dtype='object')

In [49]:
del iarc_all['doi_normalized_x']
del iarc_all['doi_unpaywall_normalized_x']
del iarc_all['doi_unpaywall_x']
del iarc_all['id_x']
del iarc_all['doi_x']
del iarc_all['doi_normalized_y']
del iarc_all['doi_unpaywall_normalized_y']
del iarc_all['doi_unpaywall_y']
del iarc_all['id_y']
del iarc_all['doi_y']
# rename columns
iarc_all = iarc_all.rename(columns={'publication_unige_x' : 'publication_doi_is_unige_publication',
                                      'publication_unige_y' : 'citation_doi_is_unige_publication',
                                      'citation_unige_x' : 'publication_doi_is_unige_citation',
                                      'citation_unige_y' : 'citation_doi_is_unige_citation',
                                     'publication_iarc_x' : 'publication_doi_is_iarc_publication',
                                      'publication_iarc_y' : 'citation_doi_is_iarc_publication',
                                      'citation_iarc_x' : 'publication_doi_is_iarc_citation',
                                      'citation_iarc_y' : 'citation_doi_is_iarc_citation',
                                     'oa_status_x' : 'publication_doi_oa_status',
                                     'oa_status_y' : 'citation_doi_oa_status',
                                     'has_repository_copy_x' : 'publication_doi_has_repository_copy',
                                     'has_repository_copy_y' : 'citation_doi_has_repository_copy'})
iarc_all

,publication_wos_id,publication_year,publication_type,publication_journal,publication_doi,publication_pmid,publication_wos_oa,citation_year,citation_journal,citation_doi,...,publication_doi_is_iarc_publication,publication_doi_is_iarc_citation,publication_doi_oa_status,publication_doi_has_repository_copy,citation_doi_is_unige_publication,citation_doi_is_unige_citation,citation_doi_is_iarc_publication,citation_doi_is_iarc_citation,citation_doi_oa_status,citation_doi_has_repository_copy
0,WOS:000492162206048,2019,Meeting Abstract,J THORAC ONCOL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WOS:000438342200039,2018,Letter,INT J EPIDEMIOL,10.1093/ije/dyy122,NaN,Bronze,2018.0,INT J EPIDEMIOL,10.1093/ije/dyy063,...,1.0,NaN,bronze,False,NaN,NaN,NaN,1.0,bronze,False
2,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2003.0,INT J EPIDEMIOL,10.1093/ije/dyg257,...,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,bronze,True
3,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2017.0,R P,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WOS:000443381100007,2018,Editorial Material,EPIDEMIOL PREV,10.19191/EP18.1.P009.008,29506354.0,NaN,2016.0,LISTEN LIBERAL WHAT,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275617,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,ATL SUST DEV GOALS 2,NaN,...,1.0,NaN,hybrid,True,NaN,NaN,NaN,NaN,NaN,NaN
275618,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2011.0,BURD BEST BUYS RED E,NaN,...,1.0,NaN,hybrid,True,NaN,NaN,NaN,NaN,NaN,NaN
275619,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2006.0,WORK TOG HLTH WORLD,NaN,...,1.0,NaN,hybrid,True,NaN,NaN,NaN,NaN,NaN,NaN
275620,WOS:000449710900010,2018,Article,LANCET,10.1016/S0140-6736(18)32281-5,30496107.0,"Green Published, Green Accepted, hybrid, Green...",2018.0,WORLD HLTH STAT 2018,NaN,...,1.0,NaN,hybrid,True,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
iarc_all.columns

Index(['publication_wos_id', 'publication_year', 'publication_type',
       'publication_journal', 'publication_doi', 'publication_pmid',
       'publication_wos_oa', 'citation_year', 'citation_journal',
       'citation_doi', 'publication_doi_is_unige_publication',
       'publication_doi_is_unige_citation',
       'publication_doi_is_iarc_publication',
       'publication_doi_is_iarc_citation', 'publication_doi_oa_status',
       'publication_doi_has_repository_copy',
       'citation_doi_is_unige_publication', 'citation_doi_is_unige_citation',
       'citation_doi_is_iarc_publication', 'citation_doi_is_iarc_citation',
       'citation_doi_oa_status', 'citation_doi_has_repository_copy'],
      dtype='object')

In [51]:
# export of dois without unpaywall information to test a new download
unige_publications_with_dois_without_unpaywall = unige_all.loc[unige_all['publication_doi'].notna() & unige_all['publication_doi_oa_status'].isna()][['publication_wos_id', 'publication_doi']]
unige_citations_with_dois_without_unpaywall = unige_all.loc[unige_all['citation_doi'].notna() & unige_all['citation_doi_oa_status'].isna()][['publication_wos_id', 'citation_doi']]
iarc_publications_with_dois_without_unpaywall = iarc_all.loc[iarc_all['publication_doi'].notna() & iarc_all['publication_doi_oa_status'].isna()][['publication_wos_id', 'publication_doi']]
iarc_citations_with_dois_without_unpaywall = iarc_all.loc[iarc_all['citation_doi'].notna() & iarc_all['citation_doi_oa_status'].isna()][['publication_wos_id', 'citation_doi']]
# rename columns
unige_publications_with_dois_without_unpaywall = unige_publications_with_dois_without_unpaywall.rename(columns={'publication_doi' : 'doi'})
unige_citations_with_dois_without_unpaywall = unige_citations_with_dois_without_unpaywall.rename(columns={'citation_doi' : 'doi'})
iarc_publications_with_dois_without_unpaywall = iarc_publications_with_dois_without_unpaywall.rename(columns={'publication_doi' : 'doi'})
iarc_citations_with_dois_without_unpaywall = iarc_citations_with_dois_without_unpaywall.rename(columns={'citation_doi' : 'doi'})

In [52]:
# append dfs and reindex
dois_without_unpaywall = unige_publications_with_dois_without_unpaywall.append(unige_citations_with_dois_without_unpaywall, ignore_index=True)
dois_without_unpaywall = dois_without_unpaywall.append(iarc_publications_with_dois_without_unpaywall, ignore_index=True)
dois_without_unpaywall = dois_without_unpaywall.append(iarc_citations_with_dois_without_unpaywall, ignore_index=True)
# dedup
dois_without_unpaywall = dois_without_unpaywall.drop_duplicates(subset='doi')
# reset index
dois_without_unpaywall.reset_index(drop=True, inplace=True)
# add new id
dois_without_unpaywall['id'] = dois_without_unpaywall.index + 800001
dois_without_unpaywall

,publication_wos_id,doi,id
0,WOS:000493407400011,10.23804/ejpd.2019.20.01.11,800001
1,WOS:000330287600007,10.3290/j.qi.a30771,800002
2,WOS:000312468300009,10.3290/j.jad.a27795,800003
3,WOS:000541911400003,10.3290/j.qi.a44635,800004
4,WOS:000329520000003,10.3290/j.qi.a29751,800005
...,...,...,...
11121,WOS:000380844300001,10.1079/PAVSNNR20094005,811122
11122,WOS:000562463000002,"[10.1016/j.atherosclerosis.2019.08.014, 10.109...",811123
11123,WOS:000529097800007,10.1101/326470V6,811124
11124,WOS:000385285000007,"[10.1016/S0140-6736(15)60401-9, 10.1016/S0140-...",811125


In [53]:
dois_without_unpaywall[['id', 'doi']].to_csv('export_dois_without_unpaywall.tsv', sep='\t', index=False)

In [54]:
# export files
unige_all.to_csv('unige_all_oa_status_citations.csv', sep='\t', index=False, encoding='utf-8')
# unige_all.to_excel('unige_all_oa_status_citations.xlsx', index=False)
iarc_all.to_csv('iarc_all_oa_status_citations.csv', sep='\t', index=False, encoding='utf-8')
iarc_all.to_excel('iarc_all_oa_status_citations.xlsx', index=False)